In [2]:
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.transformations import unwrap, center_in_box
from MDAnalysis.analysis import align
from MDAnalysis import transformations

# Load the molecular dynamics trajectory files (PD.gro and PD.xtc)
u = mda.Universe("PD.gro", "PD.xtc")

# Select the protein and DNA molecules from the trajectory
protein = u.select_atoms('protein')
dna = u.select_atoms('nucleic')

# Select the protein-DNA complex (both protein and DNA)
complex = u.select_atoms('protein or nucleic')

# Guess and add bonds between atoms in the system
u.atoms.guess_bonds()

# Create a copy of the trajectory as a reference
ref_u = u.copy()

# Select the DNA molecule from the reference trajectory
reference = ref_u.select_atoms("nucleic")

# Define a workflow of transformations to preprocess the trajectory:
workflow = (
    # Step 1: Unwrap the complex to avoid broken molecules due to periodic boundaries
    transformations.unwrap(complex),
    
    # Step 2: Center the DNA molecule in the simulation box, using its mass center
    transformations.center_in_box(dna, center='mass'),
    
    # Step 3: Wrap the complex back into the simulation box, treating fragments as units
    transformations.wrap(complex, compound='fragments'),
    
    # Step 4: Align the DNA molecule to the reference DNA using rotation and translation
    transformations.fit_rot_trans(dna, reference)
)

# Apply the defined transformations to the trajectory
u.trajectory.add_transformations(*workflow)

# Visualize the processed trajectory using nglview
view = nv.show_mdanalysis(u)
view

NGLWidget(max_frame=30009)